# Import statements

In [ ]:
import cv2
import base64
import openai

# Initialize OpenAI Client

In [ ]:
client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")

# Split video to frames

In [ ]:
# Load the cricket video
video = cv2.VideoCapture("ex1.mov")

# Calculate video length
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = video.get(cv2.CAP_PROP_FPS)
video_length_seconds = length / fps

print(f'Video length: {video_length_seconds:.2f} seconds')

# Read frames and encode to base64
base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# Release the video object
video.release()
print(len(base64Frames), "frames read.")


# Generate commentary for a single given event 

* Here we are passing evey 30th frame of a cricket video to the gpt-4o-mini model while prompting it to generate commentary in cricket commentary style 

In [ ]:
# Create OpenAI chat completion for cricket commentary
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": [
                f"These are frames of a video. Create a very short and simple voiceover script in the style of a cricket commentator for {video_length_seconds:.2f} seconds. Only include the narration. Don't talk about the view",
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::30]),
            ]
        }
    ],
    max_tokens=200,
)

# Print the generated cricket commentary
cricket_commentary = response.choices[0].message.content
print(cricket_commentary)
